# 🦜🔗 LangChain Fundamentals with Groq API

Welcome to **LangChain Fundamentals**! This notebook will teach you the core concepts of LangChain using the **free Groq API**.

## What You'll Learn:
1. **Setup & Configuration** - Environment setup with Groq API
2. **ChatGroq Model** - Initialize and use Groq's LLM
3. **Messages** - SystemMessage, HumanMessage, AIMessage
4. **Prompt Templates** - Reusable prompts with variables
5. **LCEL Chains** - Combine components with the `|` operator
6. **Output Parsers** - Parse and structure LLM responses
7. **Streaming & Batching** - Efficient response handling
8. **Tools** - Function calling with LangChain

---

## Prerequisites
- Get your free Groq API key at: https://console.groq.com/keys
- Add it to your `.env` file as `GROQ_API_KEY=your_key_here`

## 1. Setup & Installation

First, let's install and import the required packages.

In [ ]:
# Install required packages (uncomment if needed)
# %pip install langchain langchain-groq python-dotenv -q

In [ ]:
# Import required libraries
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# If .env doesn't work, you can set the API key directly:
# os.environ["GROQ_API_KEY"] = "your-groq-api-key-here"

# Verify API key is loaded
api_key = os.getenv("GROQ_API_KEY")
if not api_key:
    print("⚠️ API Key not found! Please either:")
    print("   1. Create a .env file with: GROQ_API_KEY=your-key-here")
    print("   2. Or uncomment and set the os.environ line above")
else:
    print(f"✅ API Key loaded successfully!")

## 2. Initialize ChatGroq Model

**ChatGroq** is LangChain's integration with Groq's fast inference API. 

### Available Free Groq Models:
| Model | Speed | Best For |
|-------|-------|----------|
| `llama-3.3-70b-versatile` | 280 tok/s | General purpose, high quality |
| `llama-3.1-8b-instant` | 560 tok/s | Fast responses, simpler tasks |
| `mixtral-8x7b-32768` | 450 tok/s | Complex reasoning |

## 3. Messages in LangChain

LangChain uses **message types** to structure conversations:

- **SystemMessage**: Sets the behavior/role of the AI
- **HumanMessage**: User's input
- **AIMessage**: AI's response (for conversation history)

## 4. Prompt Templates

**Prompt Templates** let you create reusable prompts with variables. This is essential for building scalable LLM applications.

## 5. LCEL Chains (LangChain Expression Language)

**LCEL** is LangChain's way to compose components using the **pipe operator `|`**. It's like Unix pipes for LLM applications!

```
prompt | llm | output_parser
```

This creates a chain where data flows: `input → prompt → llm → output_parser → output`

## 6. Output Parsers - Structured Output

Output parsers transform LLM text responses into structured data like **JSON** or **Pydantic models**.

## 7. Streaming & Batching

- **Streaming**: Get responses token-by-token (great for UX)
- **Batching**: Process multiple inputs in parallel (great for efficiency)

## 8. Tools & Function Calling

**Tools** allow the LLM to call external functions. This is the foundation for building **AI agents**.

## 🎯 Summary

In this notebook, you learned the **LangChain Fundamentals**:

| Concept | What You Learned |
|---------|------------------|
| **ChatGroq** | Initialize Groq models with temperature, max_tokens |
| **Messages** | SystemMessage, HumanMessage, AIMessage for conversations |
| **Prompt Templates** | Create reusable prompts with variables |
| **LCEL Chains** | Compose components with the `\|` operator |
| **Output Parsers** | Get structured data using Pydantic models |
| **Streaming** | Token-by-token response for better UX |
| **Batching** | Process multiple inputs efficiently |
| **Tools** | Let LLMs call external functions |

---

### Next Steps: RAG Fundamentals
In the next notebook, we'll cover **Retrieval Augmented Generation (RAG)**:
- Document loaders
- Text splitters  
- Embeddings
- Vector stores
- Retrieval chains